# Kickstarter Project

### Definition of relevant columns

* backers_count: amount of people pledging money to the project                                     
* category -> 'slug': name of the projects' specific parent- & sub-category (part of json string)
* country: country of the projects creator 
* creator -> 'id': id of the creator -> to be used as categorical variable (part of json string)
* goal: information on the amount of money needed to succeed in the local currency of the project
* launched_at: start date? of the project ()
* deadline: end date of the project ()
* spotlight: project highlighted on the website
* staff_pick: marked by a staff member of kickstarter (more attention drawn towards project)
* state: (successful/failed/canceled/live/suspended) -> exclude 'live' and combine 'canceled', 'suspended' with 'failed'
* static_usd_rate: exchange rate to transform goal in every column from current currency to USD



### Stakeholder: Project creator 
### Question: Is it useful to put much effort into launching a campaign on kickstarter? 
### Measure: Is the campaign likely to succeed or fail?

## Import Libraries

In [91]:
# Libraries

import os, json, re
import pandas as pd 



## Important Functions

In [92]:
######### functions for pre-processing ####################################################################

def extract_year_date_month(df, column):
    '''Takes a column, converts it to datetime, and creates new columns with day, month and year
    The new columns are named:
        - column_weekday
        - column_month
        - column_year
    '''
    
    # Convert column in df to datetime
    df[column] = pd.to_datetime(df[column], unit='s')

    # extract the day, month, and year components
    df[column + '_' + 'weekday'] = df[column].dt.weekday
    df[column + '_' + 'month'] = df[column].dt.month
    #df[column + '_' + 'year'] = df[column].dt.year

    return df


def duration(df, column1, column2):
    '''Returns the duration in days between 2 columns with datetime and puts it into a new colum
        - column1: start date
        - column2: end date
    '''
    df['duration_days'] = (df[column2] - df[column1]).dt.days

    return df

def convert_to_usd(df):
    return round(df['goal'] * df['static_usd_rate'],2)

######### functions for analysing predictions ########################################################## 



## Load data into one dataframe

In [93]:
directory = 'Kickstarter_data/'
data = pd.DataFrame()
relevant_columns = ['backers_count', 'category', 'country', 'creator', 'spotlight', 'staff_pick', 'state', 'static_usd_rate', 'goal', 'launched_at', 'deadline']

for file in sorted(os.listdir(directory)):
    df_temp = pd.read_csv(directory+file)
    data = pd.concat([data, df_temp[relevant_columns]], ignore_index=True)

data.head()

,backers_count,category,country,creator,spotlight,staff_pick,state,static_usd_rate,goal,launched_at,deadline
0,21,"{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""po...",US,"{""id"":1495925645,""name"":""Daniel"",""is_registere...",True,False,successful,1.000000,200.0,1388011046,1391899046
1,97,"{""id"":54,""name"":""Mixed Media"",""slug"":""art/mixe...",US,"{""id"":1175589980,""name"":""Katherine"",""slug"":""fr...",True,False,successful,1.000000,400.0,1550073611,1551801611
2,88,"{""id"":280,""name"":""Photobooks"",""slug"":""photogra...",US,"{""id"":1196856269,""name"":""MelissaThomas"",""is_re...",True,True,successful,1.000000,27224.0,1478012330,1480607930
3,193,"{""id"":266,""name"":""Footwear"",""slug"":""fashion/fo...",IT,"{""id"":1569700626,""name"":""WAO"",""slug"":""wearewao...",True,False,successful,1.136525,40000.0,1540684582,1544309940
4,20,"{""id"":51,""name"":""Software"",""slug"":""technology/...",US,"{""id"":1870845385,""name"":""Kalpit Jain"",""is_regi...",False,False,failed,1.000000,1000.0,1425919017,1428511017


## Work on the json string columns

### Extract the 'slug' parameter from the category column and drop the category column

In [94]:
cat_data = data["category"].apply(json.loads)
cat_data = pd.DataFrame(cat_data.tolist())
data['slug'] = cat_data['slug']
data = data.drop("category", axis=1)

### Extract the ID from the creator column and drop the creator column

In [95]:
data["creator_id"] = data["creator"].apply(lambda x: re.findall(r'\d+', x)[0])
data = data.drop("creator", axis=1)


## Work on the datetime columns

### Convert date-data to type date.time()

In [96]:
data['launched_at'] = pd.to_datetime(data['launched_at'], unit='s')
data['deadline'] = pd.to_datetime(data['deadline'], unit='s')

### Extract weekday and month of kickstarter project launch, as well as the duration of the kickstarter project and drop the "launched_at" and "deadline" column

In [97]:
data = extract_year_date_month(data, 'launched_at')
data = duration(data, 'launched_at', 'deadline')

data = data.drop(['launched_at', 'deadline'], axis=1)

### Convert unit of "goal" to USD and drop "static_usd_rate" and "goal" column

In [98]:
data['goal_in_usd'] = data.apply(convert_to_usd, axis=1)
data = data.drop(['static_usd_rate', 'goal'], axis=1)

In [99]:
data.head()

,backers_count,country,spotlight,staff_pick,state,slug,creator_id,launched_at_weekday,launched_at_month,duration_days,goal_in_usd
0,21,US,True,False,successful,music/rock,1495925645,2,12,45,200.0
1,97,US,True,False,successful,art/mixed media,1175589980,2,2,20,400.0
2,88,US,True,True,successful,photography/photobooks,1196856269,1,11,30,27224.0
3,193,IT,True,False,successful,fashion/footwear,1569700626,5,10,41,45461.0
4,20,US,False,False,failed,technology/software,1870845385,0,3,30,1000.0
